In [0]:
                                    # Scenario 1
                                    
# Query to get who are getting equal salary
# Input :-
# +--------+---------+--------+------+-------------------+------+
# |workerid|firstname|lastname|salary|        joiningdate|depart|
# +--------+---------+--------+------+-------------------+------+
# |     001|   Monika|   Arora|100000|2014-02-20 09:00:00|    HR|
# |     002| Niharika|   Verma|300000|2014-06-11 09:00:00| Admin|
# |     003|   Vishal| Singhal|300000|2014-02-20 09:00:00|    HR|
# |     004|  Amitabh|   Singh|500000|2014-02-20 09:00:00| Admin|
# |     005|    Vivek|   Bhati|500000|2014-06-11 09:00:00| Admin|
# +--------+---------+--------+------+-------------------+------+
# Expected Output :-
# +--------+---------+--------+------+-------------------+------+
# |workerid|firstname|lastname|salary|        joiningdate|depart|
# +--------+---------+--------+------+-------------------+------+
# |     002| Niharika|   Verma|300000|2014-06-11 09:00:00| Admin|
# |     003|   Vishal| Singhal|300000|2014-02-20 09:00:00|    HR|
# |     004|  Amitabh|   Singh|500000|2014-02-20 09:00:00| Admin|
# |     005|    Vivek|   Bhati|500000|2014-06-11 09:00:00| Admin|
# +--------+---------+--------+------+-------------------+------+
# Solution :-``


In [0]:
# dbfs:/FileStore/tables/Scenario1.csv 

DF_SC1 = spark.read.format('csv') \
    .option('header',True) \
    .option('inferschema',True) \
    .load('/FileStore/tables/Scenario1.csv')  
display(DF_SC1)

workerid,firstname,lastname,salary,joiningdate,depart
1.0,Monika,Arora,100000,2014-02-20T09:00:00Z,HR
2.0,Niharika,Verma,300000,2014-06-11T09:00:00Z,Admin
3.0,Vishal,Singhal,300000,2014-02-20T09:00:00Z,HR
4.0,Amitabh,Singh,500000,2014-02-20T09:00:00Z,Admin
5.0,Vivek,Bhati,500000,2014-06-11T09:00:00Z,Admin


In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window as W

Salary_Window = W.partitionBy('salary')
DF_Columns = DF_SC1.columns
DF_SC1 = DF_SC1.withColumn("Column_Count" , F.count('workerid').over(Salary_Window))
DF_SC1_Result = DF_SC1.select([F.col("{}".format(str(i))) for i in DF_Columns]).filter(F.col('Column_Count') > 1)
display(DF_SC1_Result)

workerid,firstname,lastname,salary,joiningdate,depart
2.0,Niharika,Verma,300000,2014-06-11T09:00:00Z,Admin
3.0,Vishal,Singhal,300000,2014-02-20T09:00:00Z,HR
4.0,Amitabh,Singh,500000,2014-02-20T09:00:00Z,Admin
5.0,Vivek,Bhati,500000,2014-06-11T09:00:00Z,Admin


In [0]:
                                        # Scenerio-2
# (Need the dates when the status gets changed like ordered to dispatched)
# Input :-
# +-------+----------+----------+
# |orderid|statusdate|    status|
# +-------+----------+----------+
# |      1|     1-Jan|   Ordered|
# |      1|     2-Jan|dispatched|
# |      1|     3-Jan|dispatched|
# |      1|     4-Jan|   Shipped|
# |      1|     5-Jan|   Shipped|
# |      1|     6-Jan| Delivered|
# |      2|     1-Jan|   Ordered|
# |      2|     2-Jan|dispatched|
# |      2|     3-Jan|   shipped|
# +-------+----------+----------+
# Expected Output :-
# +-------+----------+----------+
# |orderid|statusdate|    status|
# +-------+----------+----------+
# |      1|     2-Jan|dispatched|
# |      1|     3-Jan|dispatched|
# |      2|     2-Jan|dispatched|
# +-------+----------+----------+

In [0]:
DF_SC2 = spark.read.format('csv') \
    .option('header' , True) \
    .option('inferschema' , True) \
    .load('/FileStore/tables/Scenario2-1.csv')

display(DF_SC2)

orderid,statusdate,status
1,01-Jan,Ordered
1,02-Jan,dispatched
1,03-Jan,dispatched
1,04-Jan,Shipped
1,05-Jan,Shipped
1,06-Jan,Delivered
2,01-Jan,Ordered
2,02-Jan,dispatched
2,03-Jan,shipped


In [0]:
from pyspark.sql import functions as F 
from pyspark.sql import Window as W
from pyspark.sql.types import IntegerType

# Row_Window = W().orderBy('statusdate')
DF_Header_SC2 = DF_SC2.columns
DF_SC2 = DF_SC2.withColumn("Temp" , F.monotonically_increasing_id())
RW = W().orderBy("Temp")
DF_SC2 = DF_SC2.withColumn("Row_ID" , F.row_number().over(RW))
DF_SC2 = DF_SC2.drop("Temp")
LagW = W().orderBy("Row_ID")
DF_SC2 = DF_SC2.withColumn("Previous" , F.lag(F.col("status"),1).over(LagW))
DF_SC2_Result = DF_SC2.select([F.col("{}".format(str(i))) for i in DF_Header_SC2]) \
                .filter((F.col("status").contains('dispatched') & F.col("Previous").contains('Ordered')) | \
                (F.col("status").contains('dispatched') & F.col("Previous").contains('dispatched')))
 
display(DF_SC2_Result)

orderid,statusdate,status,Row_ID,Previous
1,02-Jan,dispatched,2,Ordered
1,03-Jan,dispatched,3,dispatched
2,02-Jan,dispatched,8,Ordered


In [0]:
result = DF_SC2.filter(
    (F.col("status") == "dispatched") &
    (F.col("orderid").isin(
        [F.col('orderid') for row in DF_SC2.filter(F.col("status") == 'Ordered').select("orderid")]
    )))

display(result)

orderid,statusdate,status,Row_ID,Previous
1,02-Jan,dispatched,2,Ordered
1,03-Jan,dispatched,3,dispatched
2,02-Jan,dispatched,8,Ordered


In [0]:
                                                        # Scenerio-3
# Input :-
# +--------+----------+------+
# |sensorid| timestamp|values|
# +--------+----------+------+
# |    1111|2021-01-15|    10|
# |    1111|2021-01-16|    15|
# |    1111|2021-01-17|    30|
# |    1112|2021-01-15|    10|
# |    1112|2021-01-15|    20|
# |    1112|2021-01-15|    30|
# +--------+----------+------+
# Expected Output :-
# +--------+----------+------+
# |sensorid| timestamp|values|
# +--------+----------+------+
# |    1111|2021-01-15|     5|
# |    1111|2021-01-16|    15|
# |    1112|2021-01-15|    10|
# |    1112|2021-01-15|    10|
# +--------+----------+------+

In [0]:
File_Path_SC3 =  "/FileStore/tables/Scenario3.csv"

DF_SC3 = spark.read.format('csv') \
    .option('header' , True) \
    .option('inferschema' , True) \
    .load(File_Path_SC3)

display(DF_SC3)

sensorid,timestamp,values
1111.0,2021-01-15,10.0
1111.0,2021-01-16,15.0
1111.0,2021-01-17,30.0
1112.0,2021-01-15,10.0
1112.0,2021-01-15,20.0
1112.0,2021-01-15,30.0


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

DF_Header_SC3 = DF_SC3.columns
DF_SC3 = DF_SC3.withColumn("ID",monotonically_increasing_id())
LeadW = W().partitionBy('sensorid').orderBy("ID") 
DF_SC3_Result = DF_SC3.withColumn("Next_values" , lead(col('values'),1).over(LeadW)) \
                .filter(col('Next_values').isNotNull()) \
                .withColumn("values" , expr("Next_values - values")) \
                .drop("Next_values") \
                .drop("ID")
    
display(DF_SC3_Result)

sensorid,timestamp,values
1111.0,2021-01-15,5.0
1111.0,2021-01-16,15.0
1112.0,2021-01-15,10.0
1112.0,2021-01-15,10.0


In [0]:
                                                # Scenerio-4
# (Write a query to list the unique customer names in the custtab table, along with the number of addresses associated with each customer.)
# Input :-
# +------+-----------+-------+
# |custid|   custname|address|
# +------+-----------+-------+
# |     1|   Mark Ray|     AB|
# |     2|Peter Smith|     CD|
# |     1|   Mark Ray|     EF|
# |     2|Peter Smith|     GH|
# |     2|Peter Smith|     CD|
# |     3|       Kate|     IJ|
# +------+-----------+-------+
# Expected Output :-
# +------+-----------+--------+
# |custid|   custname| address|
# +------+-----------+--------+
# |     1|   Mark Ray|[EF, AB]|
# |     2|Peter Smith|[CD, GH]|
# |     3|       Kate|    [IJ]|
# +------+-----------+--------+

In [0]:
data = [(1, "Mark Ray", "AB"),
        (2, "Peter Smith", "CD"),
        (1, "Mark Ray", "EF"),
        (2, "Peter Smith", "GH"),
        (2, "Peter Smith", "CD"),
        (3, "Kate", "IJ")]
myschema = ["custid", "custname", "address"]

DF_SC4 = spark.createDataFrame(data,myschema)

display(DF_SC4)

custid,custname,address
1,Mark Ray,AB
2,Peter Smith,CD
1,Mark Ray,EF
2,Peter Smith,GH
2,Peter Smith,CD
3,Kate,IJ


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

AddrW = Window().partitionBy('custname')

DF_SC4_Result = DF_SC4.withColumn("address_trf" , collect_set('address').over(AddrW)).dropDuplicates(['custname']).orderBy('custid')

DF_SC4_Result.alias('a').join(DF_SC4.alias('b') , col('a.custid') > col('b.custid') , how='inner').show()

DF_SC4_Result.show()

+------+-----------+-------+-----------+------+-----------+-------+
|custid|   custname|address|address_trf|custid|   custname|address|
+------+-----------+-------+-----------+------+-----------+-------+
|     3|       Kate|     IJ|       [IJ]|     1|   Mark Ray|     AB|
|     2|Peter Smith|     CD|   [CD, GH]|     1|   Mark Ray|     AB|
|     3|       Kate|     IJ|       [IJ]|     2|Peter Smith|     CD|
|     3|       Kate|     IJ|       [IJ]|     1|   Mark Ray|     EF|
|     2|Peter Smith|     CD|   [CD, GH]|     1|   Mark Ray|     EF|
|     3|       Kate|     IJ|       [IJ]|     2|Peter Smith|     GH|
|     3|       Kate|     IJ|       [IJ]|     2|Peter Smith|     CD|
+------+-----------+-------+-----------+------+-----------+-------+

+------+-----------+-------+-----------+
|custid|   custname|address|address_trf|
+------+-----------+-------+-----------+
|     1|   Mark Ray|     AB|   [EF, AB]|
|     2|Peter Smith|     CD|   [CD, GH]|
|     3|       Kate|     IJ|       [IJ]|
+----

In [0]:
data = [
    ("India",),
    ("Pakistan",),
    ("SriLanka",)
]
myschema = ["teams"]
df = spark.createDataFrame(data, schema=myschema)
df.show()

+--------+
|   teams|
+--------+
|   India|
|Pakistan|
|SriLanka|
+--------+



In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

joindf = df.alias("a").join(df.alias("b"), col("a.teams") < col("b.teams"), "inner")
joindf.show()

finaldf = joindf.withColumn("matches", expr("concat(a.teams,' Vs ',b.teams)")).drop("teams", "teams").show()

+--------+--------+
|   teams|   teams|
+--------+--------+
|   India|Pakistan|
|   India|SriLanka|
|Pakistan|SriLanka|
+--------+--------+

+--------------------+
|             matches|
+--------------------+
|   India Vs Pakistan|
|   India Vs SriLanka|
|Pakistan Vs SriLanka|
+--------------------+



In [0]:
                                                # Scenerio-6
# (For Employee salary greater than 10000 give designation as manager else employee)
# Input :-
# +-----+----+------+
# |empid|name|salary|
# +-----+----+------+
# |    1|   a| 10000|
# |    2|   b|  5000|
# |    3|   c| 15000|
# |    4|   d| 25000|
# |    5|   e| 50000|
# |    6|   f|  7000|
# +-----+----+------+
# Expected Output :-
# +-----+----+------+-----------+
# |empid|name|salary|Designation|
# +-----+----+------+-----------+
# |    1|   a| 10000|   Employee|
# |    2|   b|  5000|   Employee|
# |    3|   c| 15000|    Manager|
# |    4|   d| 25000|    Manager|
# |    5|   e| 50000|    Manager|
# |    6|   f|  7000|   Employee|
# +-----+----+------+-----------+

In [0]:
data1 = [
      ("1", "a", "10000"),
      ("2", "b", "5000"),
      ("3", "c", "15000"),
      ("4", "d", "25000"),
      ("5", "e", "50000"),
      ("6", "f", "7000")
]
myschema1 = ["empid","name","salary"]

DF_SC5 = spark.createDataFrame(data1,myschema1)
display(DF_SC5)

empid,name,salary
1,a,10000
2,b,5000
3,c,15000
4,d,25000
5,e,50000
6,f,7000


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

DF_SC5_Result = DF_SC5.withColumn("Designation" , when(col("salary") > 10000 , "Manager").otherwise("Employee"))
display(DF_SC5_Result)

empid,name,salary,Designation
1,a,10000,Employee
2,b,5000,Employee
3,c,15000,Manager
4,d,25000,Manager
5,e,50000,Manager
6,f,7000,Employee
